In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

params = {'axes.grid': True,
          'grid.linestyle': '--',
          }
plt.rcParams.update(params)

import myokit
i = myokit.formats.importer('cellml')

In [ ]:
def simulate_w_period(model, period, resample=True):
    p=myokit.Protocol()
    p.schedule(1, 10, 1, period, 10)
    s = myokit.Simulation(model, p)
    T = p.characteristic_time()
    d = s.run(T)
    t=np.array(d.time())
    t2 = T-period # time of last impulse
    i2 = np.where(t>t2)[0][0] # index of last impulse
    tt, V = t[i2:]-t2, np.array(d['membrane.V'][i2:])
    if resample:
        t_res = np.arange(-100, 900)
        V_res = np.interp(t_res, tt, V, left=V.min())
        return t_res+100, V_res
    else:
        return tt, V

In [ ]:
m, _, _ = myokit.load('cell_models/tt_epi.mmt'); name = 'TT-epi'
# m, _, _ = myokit.load('cell_models/lr-1991.mmt'); name = 'lr-1991'
# m, _, _ = myokit.load('cell_models/tt_endo.mmt'); name = 'TT-endo'
# m, _, _ = myokit.load('cell_models/tt_m.mmt'); name = 'TT-m'
# m, _, _ = myokit.load('cell_models/br-1977.mmt'); name = 'br-1977'
# m, _, _ = myokit.load('cell_models/decker-2009.mmt'); name = 'decker-2009'
# m, _, _ = myokit.load('cell_models/heijman-2011.mmt'); name = 'heijman-2011'
# m, _, _ = myokit.load('cell_models/ord-2011.mmt'); name = 'ord-2011'


In [ ]:
period_range =  np.logspace(np.log2(285), np.log2(10000), 10, base=2)
period_range

In [ ]:
period_range = np.array([285, 1000, 3000, 10000]) # test

In [ ]:
period_range = [285, 300, 315, 330, 350, 370, 400, 440, 500, 700, 1000, 1150, 1300, 1500, 1800, 2100, 2500, 3100, 4000, 5000, 6500, 8000, 10000]
period_range = np.unique(np.interp(np.arange(len(period_range), step=0.2), np.arange(len(period_range)), period_range).astype(int))
plt.plot(period_range, 'ko')
plt.yscale('log')

In [ ]:
myo = []
for period in period_range:
    t, V = simulate_w_period(m, period)
    myo.append(V)
    plt.plot(t, V, label='p = '+str(period))
myo = np.array(myo)

plt.legend()
plt.xlabel('Time (ms)')
plt.ylabel('V (mV)')
plt.title(name)
plt.xlim(90, 450)
# plt.savefig(f'{name}.png', dpi=300)

In [ ]:
myo.shape, period_range.shape

In [ ]:
with open('data/tt_epi.pkl', 'wb') as f:
    pickle.dump((myo, period_range), f)